#library#

In [10]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
import pickle

#pre-processing#

In [11]:
# dataset
df = pd.read_csv("/content/Dataset_B_hotel.csv")

# drop variabel
df = df.drop(columns=["Booking_ID", "arrival_year", "arrival_date"])

# drop missing value
df.dropna(inplace=True)

# output
df.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0.0,Room_Type 1,224,10,Offline,0,0,0,65.00,0,Not_Canceled
1,2,0,2,3,Not Selected,0.0,Room_Type 1,5,11,Online,0,0,0,106.68,1,Not_Canceled
2,1,0,2,1,Meal Plan 1,0.0,Room_Type 1,1,2,Online,0,0,0,60.00,0,Canceled
3,2,0,0,2,Meal Plan 1,0.0,Room_Type 1,211,5,Online,0,0,0,100.00,0,Canceled
4,2,0,1,1,Not Selected,0.0,Room_Type 1,48,4,Online,0,0,0,94.50,0,Canceled


Dataset pemesanan hotel dimuat menggunakan `pd.read_csv()` untuk mengambil data dari file CSV. Setelah data dimuat, kolom-kolom yang tidak relevan seperti "Booking_ID", "arrival_year", dan "arrival_date" dihapus menggunakan fungsi `drop()`. Langkah selanjutnya, data yang mengandung nilai kosong atau NaN dihapus dengan `dropna()` agar hanya data yang lengkap yang tersisa. Akhirnya, hasil yang telah diproses ditampilkan dengan `df.head()`

In [12]:
# missing value
print(df.isnull().sum())

no_of_adults                            0
no_of_children                          0
no_of_weekend_nights                    0
no_of_week_nights                       0
type_of_meal_plan                       0
required_car_parking_space              0
room_type_reserved                      0
lead_time                               0
arrival_month                           0
market_segment_type                     0
repeated_guest                          0
no_of_previous_cancellations            0
no_of_previous_bookings_not_canceled    0
avg_price_per_room                      0
no_of_special_requests                  0
booking_status                          0
dtype: int64


Dilakukan pengecekan terhadap data yang hilang menggunakan fungsi `df.isnull().sum()`. Hasil dari pengecekan menunjukkan bahwa tidak ada nilai yang hilang pada setiap kolom dalam dataset.

In [13]:
# encoding
label_encoder = LabelEncoder()
df['type_of_meal_plan'] = label_encoder.fit_transform(df['type_of_meal_plan'])
df['room_type_reserved'] = label_encoder.fit_transform(df['room_type_reserved'])
df['market_segment_type'] = label_encoder.fit_transform(df['market_segment_type'])
df['booking_status'] = label_encoder.fit_transform(df['booking_status'])
df.head(5)

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,0,0.0,0,224,10,3,0,0,0,65.00,0,1
1,2,0,2,3,3,0.0,0,5,11,4,0,0,0,106.68,1,1
2,1,0,2,1,0,0.0,0,1,2,4,0,0,0,60.00,0,0
3,2,0,0,2,0,0.0,0,211,5,4,0,0,0,100.00,0,0
4,2,0,1,1,3,0.0,0,48,4,4,0,0,0,94.50,0,0


Dilakukan proses encoding untuk beberapa kolom kategori, menggunakan `LabelEncoder`. Kolom-kolom yang di-encode adalah "type_of_meal_plan", "room_type_reserved", "market_segment_type", dan "booking_status". Setiap nilai kategori pada kolom-kolom tersebut diubah menjadi angka agar dapat diproses dalam model machine learning.

#training#

In [14]:
# feature & target
X = df.drop(columns=['booking_status'])
y = df['booking_status']

# test & train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# random forest
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X_train, y_train)

# xgboost
xgb_model = xgb.XGBClassifier(random_state=42)
xgb_model.fit(X_train, y_train)

# prediksi
rf_pred = rf_model.predict(X_test)
xgb_pred = xgb_model.predict(X_test)

Dilakukan pemisahan antara fitur dan target. Kolom "booking_status" dipilih sebagai target yang akan diprediksi, sementara kolom lainnya menjadi fitur. Proses ini dilakukan dengan menggunakan `drop()` untuk menghapus kolom "booking_status" dari dataset, dan nilai target disimpan dalam variabel `y`.

Selanjutnya, data dibagi menjadi dua set: data pelatihan (training) dan data pengujian (testing) dengan menggunakan `train_test_split()`, dimana 80% data digunakan untuk pelatihan dan 20% untuk pengujian.

Setelah itu, dua model machine learning diterapkan. Model pertama adalah **Random Forest**, yang dilatih menggunakan data pelatihan dengan `RandomForestClassifier`. Model kedua adalah **XGBoost**, yang juga dilatih menggunakan data pelatihan dengan `XGBClassifier`.

Akhirnya, hasil prediksi dari kedua model dievaluasi dengan menggunakan data uji (`X_test`). Prediksi dari model Random Forest disimpan dalam `rf_pred` dan prediksi dari model XGBoost disimpan dalam `xgb_pred`.

#evaluasi#

In [15]:
# random forest classification report
print("Random Forest - Classification Report:")
print(classification_report(y_test, rf_pred))

# xgboost classification report
print("\nXGBoost - Classification Report:")
print(classification_report(y_test, xgb_pred))

Random Forest - Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.80      0.84      2107
           1       0.91      0.95      0.93      4415

    accuracy                           0.90      6522
   macro avg       0.90      0.87      0.88      6522
weighted avg       0.90      0.90      0.90      6522


XGBoost - Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.79      0.83      2107
           1       0.90      0.94      0.92      4415

    accuracy                           0.89      6522
   macro avg       0.89      0.87      0.88      6522
weighted avg       0.89      0.89      0.89      6522



Dilakukan evaluasi terhadap hasil prediksi kedua model dengan berbagai metrik performa seperti **precision**, **recall**, **f1-score**, dan **support** untuk setiap kelas, serta metrik rata-rata seperti **accuracy**, **macro avg**, dan **weighted avg**.

Pertama, **Random Forest** menunjukkan bahwa model ini memiliki **precision** yang tinggi untuk kelas 0 dan 1 (0.88 dan 0.91), dengan **recall** yang juga cukup baik (0.80 dan 0.95). Nilai **f1-score** untuk kelas 1 sedikit lebih tinggi (0.93), yang menunjukkan performa yang baik dalam prediksi kelas tersebut.

Selanjutnya, **XGBoost** juga dicetak dengan hasil yang sama. **Precision** untuk kelas 0 adalah 0.87 dan untuk kelas 1 adalah 0.90, dengan **recall** untuk kelas 1 mencapai 0.94. F1-score untuk kelas 1 juga lebih tinggi (0.92), menunjukkan bahwa model ini sangat efektif dalam memprediksi kelas tersebut.

#pickle#

In [16]:
# evaluasi
rf_accuracy = accuracy_score(y_test, rf_pred)
xgb_accuracy = accuracy_score(y_test, xgb_pred)

# niai terbaik
if rf_accuracy > xgb_accuracy:
    best_model = rf_model
    print("Model Terbaik: Random Forest")
else:
    best_model = xgb_model
    print("Model Terbaik: XGBoost")

# save model menggunakan pickle
with open('best_model.pkl', 'wb') as model_file:
    pickle.dump(best_model, model_file)

Model Terbaik: Random Forest


Pada kode ini, dilakukan perbandingan antara kedua nilai **accuracy** untuk menentukan model mana yang lebih baik. Jika **accuracy** dari Random Forest lebih tinggi, maka model terbaik adalah Random Forest, dan sebaliknya jika **accuracy** XGBoost lebih tinggi. Setelah menentukan model terbaik, kode ini kemudian menyimpan model tersebut menggunakan **pickle** dalam file dengan nama 'best_model.pkl'.